In [1]:
import pandas as pd
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define file paths
github_url = "https://raw.githubusercontent.com/apownukepcc/datathon-spring-2025/main/combined_data.csv"
output_so2tons_file = "/content/drive/My Drive/SO2TONS_dataset.csv"
output_noxtons_file = "/content/drive/My Drive/NOXTONS_dataset.csv"
output_cotons_file = "/content/drive/My Drive/COTONS_dataset.csv"

# Load the dataset from GitHub
try:
    data = pd.read_csv(github_url)
    print("Dataset successfully loaded from GitHub.")
except Exception as e:
    print(f"Error loading dataset from GitHub: {e}")
    exit()

# Define the emissions parameters
emissions_params = ['SO2TONS', 'NOXTONS', 'COTONS']
load_param = 'LOADMWBA'

# Step 1: Separate emissions and load data
emissions_data = data[data['Parameter'].isin(emissions_params)]
load_data = data[data['Parameter'] == load_param]

# Step 2: Merge emissions and load data based on 'date' and 'Source'
merged_data = pd.merge(
    emissions_data,
    load_data,
    on=["date", "Source"],
    suffixes=("_emission", "_load"),
    how="left"
)

# Step 3: Calculate Emissions_Load
merged_data["Emissions_Load"] = merged_data["Value_emission"] / merged_data["Value_load"]

# Step 4: Add Emissions_Load back to the original dataset
data_with_emissions_load = pd.merge(
    data,
    merged_data[["date", "Source", "Parameter_emission", "Emissions_Load"]],
    left_on=["date", "Source", "Parameter"],
    right_on=["date", "Source", "Parameter_emission"],
    how="left"
).drop(columns=["Parameter_emission"], errors="ignore")

# Step 5: Define a function to filter the dataset for a specific parameter
def filter_dataset(data, parameter):
    # Filter rows for the specific parameter
    parameter_data = data[data['Parameter'] == parameter]
    # Remove rows with NaN or zero Emissions_Load
    parameter_data = parameter_data.dropna(subset=['Emissions_Load'])
    parameter_data = parameter_data[parameter_data['Emissions_Load'] > 0]
    return parameter_data

# Step 6: Generate datasets for SO2TONS, NOXTONS, and COTONS
so2tons_data = filter_dataset(data_with_emissions_load, 'SO2TONS')
noxtons_data = filter_dataset(data_with_emissions_load, 'NOXTONS')
cotons_data = filter_dataset(data_with_emissions_load, 'COTONS')

# Step 7: Save the filtered datasets to separate CSV files
so2tons_data.to_csv(output_so2tons_file, index=False)
noxtons_data.to_csv(output_noxtons_file, index=False)
cotons_data.to_csv(output_cotons_file, index=False)

print(f"Datasets generated and saved:\n- {output_so2tons_file}\n- {output_noxtons_file}\n- {output_cotons_file}")


Mounted at /content/drive
Dataset successfully loaded from GitHub.
Datasets generated and saved:
- /content/drive/My Drive/SO2TONS_dataset.csv
- /content/drive/My Drive/NOXTONS_dataset.csv
- /content/drive/My Drive/COTONS_dataset.csv
